In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
path = './data/test4/'
train = pd.read_csv(path + 'train.csv' )
test = pd.read_csv(path + 'test.csv')

In [4]:
print(train.columns)
print(test.columns)

Index(['ID', 'Gender', 'Ever_Married', 'Age', 'Graduated', 'Profession',
       'Work_Experience', 'Spending_Score', 'Family_Size', 'Var_1',
       'Segmentation'],
      dtype='object')
Index(['ID', 'Gender', 'Ever_Married', 'Age', 'Graduated', 'Profession',
       'Work_Experience', 'Spending_Score', 'Family_Size', 'Var_1'],
      dtype='object')


## Preprocessing (train)

In [5]:
# train
X = train.drop(['ID','Segmentation'], axis=1)
y = train['Segmentation']

In [6]:
# 원-핫 인코딩
X1 = pd.get_dummies(X)
X1.head()

,Age,Work_Experience,Family_Size,Gender_Female,Gender_Male,Ever_Married_No,Ever_Married_Yes,Graduated_No,Graduated_Yes,Profession_Artist,...,Spending_Score_Average,Spending_Score_High,Spending_Score_Low,Var_1_Cat_1,Var_1_Cat_2,Var_1_Cat_3,Var_1_Cat_4,Var_1_Cat_5,Var_1_Cat_6,Var_1_Cat_7
0,22,1.0,4.0,0,1,1,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0
1,67,1.0,1.0,1,0,0,1,0,1,0,...,0,0,1,0,0,0,0,0,1,0
2,67,0.0,2.0,0,1,0,1,0,1,0,...,0,1,0,0,0,0,0,0,1,0
3,56,0.0,2.0,0,1,0,1,1,0,1,...,1,0,0,0,0,0,0,0,1,0
4,32,1.0,3.0,0,1,1,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0


In [7]:
X1

,Age,Work_Experience,Family_Size,Gender_Female,Gender_Male,Ever_Married_No,Ever_Married_Yes,Graduated_No,Graduated_Yes,Profession_Artist,...,Spending_Score_Average,Spending_Score_High,Spending_Score_Low,Var_1_Cat_1,Var_1_Cat_2,Var_1_Cat_3,Var_1_Cat_4,Var_1_Cat_5,Var_1_Cat_6,Var_1_Cat_7
0,22,1.0,4.0,0,1,1,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0
1,67,1.0,1.0,1,0,0,1,0,1,0,...,0,0,1,0,0,0,0,0,1,0
2,67,0.0,2.0,0,1,0,1,0,1,0,...,0,1,0,0,0,0,0,0,1,0
3,56,0.0,2.0,0,1,0,1,1,0,1,...,1,0,0,0,0,0,0,0,1,0
4,32,1.0,3.0,0,1,1,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6660,41,0.0,5.0,0,1,0,1,0,1,1,...,0,1,0,0,0,0,0,0,1,0
6661,35,3.0,4.0,0,1,1,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0
6662,33,1.0,1.0,1,0,1,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0
6663,27,1.0,4.0,1,0,1,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0


In [14]:
import warnings
warnings.filterwarnings(action='ignore')

# train test split
from sklearn.model_selection import train_test_split

# scaling
from sklearn.preprocessing import MinMaxScaler

# 분류 모델
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier # 렌담포레스트 성능개선 해보자.
from sklearn.ensemble import GradientBoostingClassifier

# scaling + pipeline
from sklearn.pipeline import Pipeline

# 교차검증 및 매개변수 튜닝
from sklearn.model_selection import GridSearchCV

# 분류모델 평가
from sklearn.metrics import classification_report

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.3,
                                                   stratify=y, random_state=1234)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(4665, 28)
(2000, 28)
(4665,)
(2000,)


### XGB

In [11]:
# 시험환경과 버전 동일하게 설치
# !pip install xgboost==1.4.2

     --------------------------------------- 97.8/97.8 MB 21.1 MB/s eta 0:00:00


In [17]:
import xgboost as xgb
print(xgb.__version__)

1.4.2


In [18]:
pipe_model = Pipeline( [('scaler', MinMaxScaler()),
                        ('model', xgb.XGBClassifier())] )

pipe_model

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('model',
                 XGBClassifier(base_score=None, booster=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, gamma=None, gpu_id=None,
                               importance_type='gain',
                               interaction_constraints=None, learning_rate=None,
                               max_delta_step=None, max_depth=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=100,
                               n_jobs=None, num_parallel_tree=None,
                               random_state=None, reg_alpha=None,
                               reg_lambda=None, scale_pos_weight=None,
                               subsample=None, tree_method=None,
                               validate_parameters=None, verbosity=None))])

In [49]:
# 매개변수 튜닝1 - 결과: 0.32509
parameter_list = {'model__n_estimators':[50,100,150],
                  'model__max_depth':[3,4,5,6],
                  'model__min_samples_split':range(5,10),
                  }

In [50]:
parameter_list

{'model__n_estimators': [50, 100, 150],
 'model__max_depth': [3, 4, 5, 6],
 'model__min_samples_split': range(5, 10)}

In [52]:
import sklearn
sklearn.metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'matthews_corrcoef', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_s

In [53]:
import time
start = time.time()  # 시작 시간 저장

grid_model = GridSearchCV(pipe_model, param_grid = parameter_list,
                          cv=5, n_jobs=-1, scoring='f1_macro', verbose=1)
grid_model.fit(X_train, y_train)

print("time :", time.time() - start) 

Fitting 5 folds for each of 60 candidates, totalling 300 fits
[13:49:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "min_samples_split" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:49:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
time : 93.75107192993164


In [56]:
best_model = grid_model.best_estimator_

In [57]:
best_model

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('model',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, gpu_id=-1,
                               importance_type='gain',
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=4, min_child_weight=1,
                               min_samples_split=5, missing=nan,
                               monotone_constraints='()', n_estimators=50,
                               n_jobs=8, num_parallel_tree=1,
                               objective='multi:softprob', random_state=0,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
                               subsample=1, tree_method='exact',
                               validate_parameters=1, verbosity=None))])

In [26]:
# 평가
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

In [27]:
# cv=3
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           1       0.53      0.56      0.54      1131
           2       0.50      0.37      0.43      1100
           3       0.61      0.66      0.63      1204
           4       0.68      0.75      0.71      1230

    accuracy                           0.59      4665
   macro avg       0.58      0.58      0.58      4665
weighted avg       0.58      0.59      0.58      4665



In [28]:
# cv=3
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           1       0.48      0.50      0.49       485
           2       0.43      0.31      0.36       472
           3       0.56      0.61      0.59       516
           4       0.64      0.72      0.68       527

    accuracy                           0.54      2000
   macro avg       0.53      0.53      0.53      2000
weighted avg       0.53      0.54      0.53      2000



## Preprocessing (Test)

In [29]:
test_ID = test.pop('ID') # 한번만 실행
test.columns

Index(['Gender', 'Ever_Married', 'Age', 'Graduated', 'Profession',
       'Work_Experience', 'Spending_Score', 'Family_Size', 'Var_1'],
      dtype='object')

In [30]:
test_ID

0       458989
1       458994
2       459000
3       459003
4       459005
         ...  
2149    467950
2150    467954
2151    467958
2152    467961
2153    467968
Name: ID, Length: 2154, dtype: int64

In [31]:
test = pd.get_dummies(test)

In [32]:
pred = best_model.predict(test)
pred

array([1, 3, 3, ..., 1, 2, 4], dtype=int64)

In [33]:
submit = pd.DataFrame({'ID': test_ID,
                       'Segmentation': pred})

submit.to_csv('./submission.csv', index=False)

### cv=5

In [58]:
# cv=5
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           1       0.48      0.50      0.49       485
           2       0.43      0.31      0.36       472
           3       0.56      0.61      0.59       516
           4       0.64      0.72      0.68       527

    accuracy                           0.54      2000
   macro avg       0.53      0.53      0.53      2000
weighted avg       0.53      0.54      0.53      2000



In [60]:
test = pd.get_dummies(test)
pred_cv5 = best_model.predict(test)
pred_cv5

array([1, 1, 3, ..., 1, 2, 4], dtype=int64)

In [61]:
submit = pd.DataFrame({'ID': test_ID,
                       'Segmentation': pred_cv5})

submit.to_csv('./submission2.csv', index=False)

In [ ]:
# result : 0.32111
# cv=3일때보다 더 떨어짐을 확인

### Try and Error

In [62]:
xgb_model = xgb.XGBClassifier(max_depth=8, n_esitimators=500)

In [63]:
xgb_model

XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=8,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_esitimators=500, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method=None, validate_parameters=None, verbosity=None)

In [66]:
# 교차 검증
from sklearn.model_selection import cross_val_score
scores = cross_val_score(xgb_model, X_train, y_train, scoring='f1_macro', cv=5)

print(scores)
print(scores.mean())

[13:56:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "n_esitimators" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:56:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:56:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "n_esitimators" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters ar

In [67]:
xgb_model.fit(X_train, y_train)
pred_3 = xgb_model.predict(test)
pred_3

[13:58:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "n_esitimators" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:58:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


array([2, 3, 3, ..., 2, 2, 4], dtype=int64)

In [72]:
y_test_pred = xgb_model.predict(X_test)
y_train_pred = xgb_model.predict(X_train)

In [73]:
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           1       0.94      0.91      0.92      1131
           2       0.91      0.86      0.88      1100
           3       0.88      0.94      0.91      1204
           4       0.96      0.97      0.96      1230

    accuracy                           0.92      4665
   macro avg       0.92      0.92      0.92      4665
weighted avg       0.92      0.92      0.92      4665



In [71]:
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           1       0.44      0.41      0.43       485
           2       0.41      0.37      0.39       472
           3       0.53      0.54      0.54       516
           4       0.61      0.70      0.65       527

    accuracy                           0.51      2000
   macro avg       0.50      0.51      0.50      2000
weighted avg       0.50      0.51      0.51      2000



In [68]:
submit = pd.DataFrame({
    'ID': test_ID,
    'Segmentation': pred_3
})
submit.to_csv("submission3.csv", index=False)
# Score: 0.32046

### Try2

In [95]:
xgb_model = xgb.XGBClassifier(max_depth=6, random_state=1234)

In [96]:
xgb_model

XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=6,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              random_state=1234, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=None, tree_method=None,
              validate_parameters=None, verbosity=None)

In [97]:
xgb_model.fit(X_train, y_train)
pred_4 = xgb_model.predict(test)
pred_4

[14:09:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


array([2, 3, 3, ..., 1, 2, 4], dtype=int64)

In [98]:
y_test_pred = xgb_model.predict(X_test)
y_train_pred = xgb_model.predict(X_train)

In [99]:
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           1       0.83      0.81      0.82      1131
           2       0.80      0.72      0.76      1100
           3       0.79      0.84      0.81      1204
           4       0.88      0.90      0.89      1230

    accuracy                           0.82      4665
   macro avg       0.82      0.82      0.82      4665
weighted avg       0.82      0.82      0.82      4665



In [101]:
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           1       0.45      0.43      0.44       485
           2       0.42      0.37      0.39       472
           3       0.55      0.56      0.56       516
           4       0.62      0.71      0.66       527

    accuracy                           0.52      2000
   macro avg       0.51      0.52      0.51      2000
weighted avg       0.51      0.52      0.52      2000



In [83]:
submit = pd.DataFrame({
    'ID': test_ID,
    'Segmentation': pred_4
})
submit.to_csv("submission4.csv", index=False)

### LGBM

In [104]:
from lightgbm import LGBMClassifier

In [106]:
lgbm = LGBMClassifier(random_state=1234)
lgbm

LGBMClassifier(random_state=1234)

In [107]:
lgbm.fit(X_train, y_train)

LGBMClassifier(random_state=1234)

In [110]:
scaler = MinMaxScaler()
X_train_ = scaler.fit_transform(X_train)

array([[0.1971831 , 0.14285714, 0.125     , ..., 0.        , 0.        ,
        0.        ],
       [0.74647887, 0.07142857, 0.375     , ..., 0.        , 1.        ,
        0.        ],
       [0.56338028, 0.07142857, 0.        , ..., 0.        , 1.        ,
        0.        ],
       ...,
       [0.30985915, 0.57142857, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.12676056, 0.07142857, 0.5       , ..., 0.        , 0.        ,
        0.        ],
       [0.14084507, 1.        , 0.        , ..., 0.        , 1.        ,
        0.        ]])

In [114]:
y_train_pred = lgbm.predict(X_train)
y_test_pred = lgbm.predict(X_test)

In [115]:
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           1       0.46      0.43      0.45       485
           2       0.42      0.38      0.40       472
           3       0.55      0.55      0.55       516
           4       0.63      0.73      0.67       527

    accuracy                           0.53      2000
   macro avg       0.52      0.52      0.52      2000
weighted avg       0.52      0.53      0.52      2000



In [118]:
pred_5 = lgbm.predict(test)

In [119]:
submit = pd.DataFrame({
    'ID': test_ID,
    'Segmentation':pred_5
})
submit.to_csv("submission5.csv", index=False)

### LGBM + Parameter Tunning

In [121]:
from sklearn.pipeline import Pipeline

In [122]:
pipe_model = Pipeline( [('scaler', MinMaxScaler()), 
                        ('model', LGBMClassifier())] )

pipe_model

Pipeline(steps=[('scaler', MinMaxScaler()), ('model', LGBMClassifier())])

In [ ]:
parameter_list = {'model__n_estimators':[50, 100, 150],
                  'model__max_depth':[4,5,6,7,8],
                  }

In [124]:
parameter_list

{'model__n_estimators': [50, 100, 150], 'model__max_depth': [4, 5, 6, 7, 8]}

In [126]:
import time
start_time = time.time()
grid_model = GridSearchCV(pipe_model, param_grid=parameter_list, cv=3)
grid_model.fit(X_train, y_train)
print('time: ', time.time() - start_time)

time:  6.093055725097656


In [129]:
best_model = grid_model.best_estimator_

In [137]:
best_model

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('model', LGBMClassifier(max_depth=5, n_estimators=50))])

In [130]:
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

In [132]:
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           1       0.57      0.57      0.57      1131
           2       0.55      0.43      0.48      1100
           3       0.64      0.67      0.65      1204
           4       0.68      0.78      0.73      1230

    accuracy                           0.62      4665
   macro avg       0.61      0.61      0.61      4665
weighted avg       0.61      0.62      0.61      4665



In [131]:
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           1       0.49      0.48      0.48       485
           2       0.45      0.36      0.40       472
           3       0.58      0.58      0.58       516
           4       0.63      0.75      0.68       527

    accuracy                           0.55      2000
   macro avg       0.54      0.54      0.54      2000
weighted avg       0.54      0.55      0.54      2000



In [133]:
pred_6 = best_model.predict(test)

In [136]:
submit = pd.DataFrame({
    'ID': test_ID,
    'Segmentation': pred_6
})
submit.to_csv('./submission6.csv', index=False)

### LGBM3

In [138]:
parameter_list2 = {'max_depth':5,
                   'n_estimators':50,
                   'booster':'dart'}

In [140]:
lgbm = LGBMClassifier(**parameter_list2)

In [153]:
num_cols = X_train.columns[:3]
scaler = MinMaxScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_train.head()

,Age,Work_Experience,Family_Size,Gender_Female,Gender_Male,Ever_Married_No,Ever_Married_Yes,Graduated_No,Graduated_Yes,Profession_Artist,...,Spending_Score_Average,Spending_Score_High,Spending_Score_Low,Var_1_Cat_1,Var_1_Cat_2,Var_1_Cat_3,Var_1_Cat_4,Var_1_Cat_5,Var_1_Cat_6,Var_1_Cat_7
6304,0.197183,0.142857,0.125,0,1,0,1,1,0,0,...,0,0,1,0,0,1,0,0,0,0
5853,0.746479,0.071429,0.375,0,1,0,1,0,1,0,...,0,1,0,0,0,0,0,0,1,0
3773,0.563380,0.071429,0.000,1,0,1,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0
3504,0.211268,1.000000,0.000,0,1,1,0,1,0,0,...,0,0,1,0,0,0,0,0,1,0
680,0.000000,0.000000,0.625,0,1,1,0,1,0,0,...,0,0,1,0,0,0,0,0,1,0


In [157]:
lgbm.fit(X_train, y_train)

LGBMClassifier(booster='dart', max_depth=5, n_estimators=50)

In [158]:
y_train_pred = lgbm.predict(X_train)
y_test_pred = lgbm.predict(X_test)

In [161]:
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           1       0.57      0.57      0.57      1131
           2       0.55      0.43      0.48      1100
           3       0.64      0.67      0.65      1204
           4       0.68      0.78      0.73      1230

    accuracy                           0.62      4665
   macro avg       0.61      0.61      0.61      4665
weighted avg       0.61      0.62      0.61      4665



In [162]:
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           1       0.35      0.68      0.47       485
           2       0.00      0.00      0.00       472
           3       0.48      0.61      0.54       516
           4       0.70      0.53      0.60       527

    accuracy                           0.46      2000
   macro avg       0.38      0.46      0.40      2000
weighted avg       0.39      0.46      0.41      2000

